# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import nltk
import re
nltk.download(['punkt','wordnet'])
import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
from sqlalchemy import create_engine
import pandas as pd
import numpy as np

from sklearn.pipeline import Pipeline

from nltk.tokenize import word_tokenize, WhitespaceTokenizer

from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.multioutput import MultiOutputClassifier

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.metrics import classification_report, accuracy_score
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC

In [3]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table('messages',engine) 
X = df['message']
y = df.iloc[:,4:]
categories=y.columns                    

In [4]:
df[~(df.isnull().any(axis=1))|(df.original.isnull())].shape, df[~(df.isnull().any(axis=1))].shape

((26180, 40), (10153, 40))

In [5]:
df=df[~(df.isnull().any(axis=1))|((df.original.isnull())&~(df.offer.isnull()))]

### 2. Write a tokenization function to process your text data

In [6]:
def tokenize(text):
    # get tokens from text
    tokens= WhitespaceTokenizer().tokenize(text)
    lemmatizer= WordNetLemmatizer()
    
    # clean tokens
    processed_tokens=[]
    for token in tokens:
        token=lemmatizer.lemmatize(token).lower().strip('!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~')
        token=re.sub(r'\[[^.,;:]]*\]','', token)
        
        # add token to compiled list if not empty
        if token !='':
            processed_tokens.append(token)
    return processed_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [7]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [8]:
import random
random.seed(20181022)

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.2)

In [10]:
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [11]:
y_pred=pipeline.predict(X_test)

In [12]:
print(classification_report(y_test.iloc[:,1:].values, np.array([x[1:] for x in y_pred]), target_names=categories))

                        precision    recall  f1-score   support

               related       0.81      0.40      0.54       895
               request       0.00      0.00      0.00        22
                 offer       0.74      0.53      0.62      2186
           aid_related       0.70      0.09      0.16       438
          medical_help       0.71      0.07      0.13       275
      medical_products       0.33      0.02      0.03       130
     search_and_rescue       0.00      0.00      0.00        80
              security       0.59      0.05      0.10       193
              military       0.00      0.00      0.00         0
           child_alone       0.83      0.24      0.37       369
                 water       0.82      0.38      0.52       564
                  food       0.81      0.18      0.30       501
               shelter       0.75      0.07      0.12        89
              clothing       0.88      0.06      0.11       118
                 money       0.00      

### 6. Improve your model
Use grid search to find better parameters. 

In [13]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(OneVsRestClassifier(LinearSVC())))
])

parameters = {
    'tfidf__smooth_idf':[True, False],
    'clf__estimator__estimator__C':[1,2,5]}

cv = GridSearchCV(pipeline, parameters)
cv.fit(X_train, y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...ty='l2', random_state=None, tol=0.0001,
     verbose=0),
          n_jobs=1),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'tfidf__smooth_idf': [True, False], 'clf__estimator__estimator__C': [1, 2, 5]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [14]:
cv.cv_results_

{'mean_fit_time': array([ 5.78378352,  5.79573925,  6.83461253,  6.82776475,  9.68571854,
         9.78559049]),
 'std_fit_time': array([ 0.04439364,  0.05685146,  0.05924221,  0.05597879,  0.03698342,
         0.21409843]),
 'mean_score_time': array([ 1.65128056,  1.65053622,  1.66311812,  1.65520684,  1.65739497,
         1.65662996]),
 'std_score_time': array([ 0.0313212 ,  0.01840314,  0.02585312,  0.0254283 ,  0.0251935 ,
         0.02020434]),
 'param_clf__estimator__estimator__C': masked_array(data = [1 1 2 2 5 5],
              mask = [False False False False False False],
        fill_value = ?),
 'param_tfidf__smooth_idf': masked_array(data = [True False True False True False],
              mask = [False False False False False False],
        fill_value = ?),
 'params': [{'clf__estimator__estimator__C': 1, 'tfidf__smooth_idf': True},
  {'clf__estimator__estimator__C': 1, 'tfidf__smooth_idf': False},
  {'clf__estimator__estimator__C': 2, 'tfidf__smooth_idf': True},
  {'clf__

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [15]:
y_pred=cv.predict(X_test)
print(accuracy_score(y_test.iloc[:,1:].values, np.array([x[1:] for x in y_pred])))

0.408517952636


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [16]:
cv.best_params_

{'clf__estimator__estimator__C': 1, 'tfidf__smooth_idf': True}

### 9. Export your model as a pickle file

In [17]:
import pickle
pickle.dump(cv,open('model.p','wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.